In [1]:
!pip install selenium webdriver-manager

   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.5 MB 4.8 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/9.5 MB 5.3 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/9.5 MB 4.4 MB/s eta 0:00:02
   ---------------- ----------------------- 3.9/9.5 MB 4.8 MB/s eta 0:00:02
   ----------------------- ---------------- 5.5/9.5 MB 5.4 MB/s eta 0:00:01
   -------------------------------- ------- 7.9/9.5 MB 6.5 MB/s eta 0:00:01
   ---------------------------------------  9.4/9.5 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------- 9.5/9.5 MB 6.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import traceback

# Optional: Run in visible mode for debugging (comment out for headless)
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Uncomment to run in headless mode

# Initialize Chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

try:
    # Step 1: Open login page
    driver.get("https://arms.sse.saveetha.com/")
    print("[INFO] Opened login page.")

    # Step 2: Fill credentials and login
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "txtusername"))).send_keys("192221048")
    driver.find_element(By.ID, "txtpassword").send_keys("suderson14@")
    driver.find_element(By.ID, "btnlogin").click()
    print("[INFO] Submitted login form.")

    # Step 3: Wait for login to complete & check current URL
    time.sleep(5)  # Allow redirect time
    current_url = driver.current_url
    print(f"[DEBUG] Current URL after login: {current_url}")

    # If we land on Landing.aspx, navigate to MyCourse.aspx
    if "Landing.aspx" in current_url:
        print("[INFO] Redirected to Landing page. Navigating to My Course...")

        # Find and click the 'My Course' button (Update if different)
        try:
            my_course_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.LINK_TEXT, "My Course"))  # Update selector if necessary
            )
            my_course_btn.click()
            print("[INFO] Clicked on 'My Course'.")
        except Exception as e:
            print(f"[ERROR] Could not find 'My Course' button. {e}")
            driver.quit()
            exit()

        # Wait for MyCourse.aspx to load
        WebDriverWait(driver, 20).until(EC.url_contains("MyCourse.aspx"))
        print("[INFO] Successfully navigated to My Course page.")

    # Step 4: Wait for grades table to load
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "tblGridViewComplete")))
    print("[INFO] Found the grades table.")

    # **FIX: Wait extra 3-5 seconds to allow table data to load**
    time.sleep(5)

    # Step 5: Extract passed courses
    passed_courses = []
    grade_point_mapping = {"S": 10, "A": 9, "B": 8, "C": 7, "D": 6, "E": 5}
    
    # **Ensure table rows are present before fetching data**
    rows = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#tblGridViewComplete tbody tr"))
    )

    total_grade_points = 0
    total_courses = 0

    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        
        # Ensure the row has enough columns
        if len(cells) < 6:
            continue
        
        status = cells[4].find_element(By.TAG_NAME, "span").text.strip()
        grade = cells[3].text.strip()

        if status.upper() == "PASS" and grade in grade_point_mapping:
            passed_courses.append({
                "Course Code": cells[1].text.strip(),
                "Course Name": cells[2].text.strip(),
                "Grade": grade,
                "Grade Points": grade_point_mapping[grade],
                "Month & Year": cells[5].text.strip()
            })
            total_grade_points += grade_point_mapping[grade]
            total_courses += 1

    # Step 6: Print results
    print(f"\n[INFO] Found {len(passed_courses)} passed courses:\n")
    for index, course in enumerate(passed_courses, 1):
        print(f"{index}. {course['Course Code']} - {course['Course Name']}")
        print(f"   Grade: {course['Grade']} (Grade Points: {course['Grade Points']}) | Completed: {course['Month & Year']}\n")

    # Step 7: Calculate and print CGPA
    if total_courses > 0:
        cgpa = total_grade_points / total_courses
        print(f"\n[INFO] CGPA: {cgpa:.2f}")
    else:
        print("\n[INFO] No passed courses found to calculate CGPA.")

except Exception as e:
    print(f"[ERROR] An unexpected error occurred: {str(e)}")
    traceback.print_exc()

finally:
    driver.quit()
    print("[INFO] Browser closed.")


[INFO] Opened login page.
[INFO] Submitted login form.
[DEBUG] Current URL after login: https://arms.sse.saveetha.com/StudentPortal/Landing.aspx
[INFO] Redirected to Landing page. Navigating to My Course...
[INFO] Clicked on 'My Course'.
[INFO] Successfully navigated to My Course page.
[INFO] Found the grades table.

[INFO] Found 29 passed courses:

1. CSA14 - Compiler design
   Grade: B (Grade Points: 8) | Completed: January-2025

2. CSA12 - Computer Architecture
   Grade: B (Grade Points: 8) | Completed: January-2025

3. ITA02 - Web Technology
   Grade: A (Grade Points: 9) | Completed: January-2025

4. ITA04 - Statistics with R Programming
   Grade: A (Grade Points: 9) | Completed: November-2024

5. CSA37 - Software Testing
   Grade: A (Grade Points: 9) | Completed: November-2024

6. CSA09 - Programming in Java
   Grade: S (Grade Points: 10) | Completed: September-2024

7. CSA11 - Object Oriented Analysis and Design
   Grade: A (Grade Points: 9) | Completed: September-2024

8. CSA15 